# Notebook Utilities

Use the NotebookLoader to import this notebook into other notebooks.

# sh()

A "run this shell command" method that wraps subprocess.check_output()

In [ ]:
import subprocess

def sh(shell_command, shell=True, stderr=None):
    try:
        return_value = subprocess.check_output(shell_command, shell=shell, stderr=stderr)
    except Exception, err:
        print('Shell command failed: {}', shell_command)
        print(err)
        return_value = 'Error'
    return return_value

# print_sh()

Same as _sh(), except that it also prints the command output, preceded by an optional label.

In [ ]:
def print_sh(shell_command, label=None, **kwargs):
    print('{0}: {1}\n'.format(label+':' if label else '', sh(shell_command, **kwargs)))

# install_driver_registry_csv()

Install a CSV-formatted registry configuration for a device driver.

In [ ]:
def install_driver_registry_csv(store=None, driver_root=None, name=None, csv=None):
    sh('volttron-ctl config store {0} {1} {2} --csv'.format(store, name, driver_root + csv))

# install_driver_config

Install a driver configuration for a device driver.

In [ ]:
def install_driver_config(store=None, driver_root=None, name=None, config=None):
    sh('volttron-ctl config store {0} {1} {2}'.format(store, name, driver_root + config))

# install_agent()

Install a VOLTTRON agent given its directory, id, and config file

In [ ]:
def install_agent(dir=None, id=None, config=None):
    script_install_command = 'python scripts/install-agent.py -s {0} -i {1} -c {2} -t {3} -f --start'
    sh(script_install_command.format(dir, id, config, id), stderr=subprocess.STDOUT)
    print('Installed and started {} with config {}'.format(id, config))

# run_sqlite_cmd()

Execute a SQLite command on the indicated database

In [ ]:
def run_sqlite_cmd(db_path, command_string):
    print_sh('sqlite3 {0} {1}'.format(db_path, command_string))

# send_vtn_event()

Send an event as though it were from a VTN

In [ ]:
import isodate
import gevent
import requests
from datetime import timedelta, datetime
from volttron.platform import get_address, get_services_core
from volttron.platform.agent import utils
from volttron.platform.control import KeyStore, KnownHostsStore
from volttron.platform.vip.agent import Agent

def send_vtn_event(vroot, web_address, web_port, event_id, start_delay, event_duration):
    if 'vtn_agent' not in globals().keys():
        global vtn_agent
        vtn_agent = Agent(identity='rpc.caller', publickey=KeyStore().public, address=get_address(),
                          secretkey=KeyStore().secret, serverkey=KnownHostsStore().serverkey(get_address()),
                          enable_store=False, enable_channel=True)
        event = gevent.event.Event()
        greenlet = gevent.spawn(vtn_agent.core.run, event)
        event.wait()

    sim_time = vtn_agent.vip.rpc.call('simulationclock', 'get_time').get(timeout=10)
    ts = isodate.strftime(datetime.strptime(sim_time, '%Y-%m-%d %H:%M:%S')+timedelta(seconds=start_delay), isodate.DT_EXT_COMPLETE)+'Z'
    print(ts)

    eid = event_id
    duration = isodate.duration_isoformat(timedelta(seconds=event_duration))
    with open(vroot+'/services/core/OpenADRVenAgent/test/xml/test_vtn_distribute_event_variable.xml', 'rb') as xml_file:
        xml_string = xml_file.read()
        xml_string = xml_string.format(event_id=event_id, event_start_time=ts, event_duration=duration)
        requests.post('http://{}:{}/OpenADR2/Simple/2.0b/EiEvent'.format(web_address, web_port),
                      data=xml_string,
                      headers={'content-type': 'application/xml'})